# Centralized Solution of Multi-Regional Transmission Expansion Horizontal Investment Coordination Problem

This notebook contains code for a three zone test case. The case consists of an artificial merging of IEEE 14 nodes, IEEE 30 nodes, and a 5 node test system, respectively, for each zone. This notebook assumes there is a single entity (e.g. national planner) to solve the optimization problem centrally.
Results from this notebook serve a benchmark for the decentralized algorithms. 


In [117]:
# Load the different package dependencies for Julia
# Load or comment out the appropriate optimization engine (E.g. Gurobi, GLPK)

import Pkg
using Pkg
using JuMP
using GLPK
using HiGHS
#using Gurobi
#using CPLEX
#using Xpress
#using Cbc
#using Clp
using XLSX
using DataFrames
using PrettyPrint

## Problem set up

Set up the example problem by reading the input data files from the `../Input_Data/` folder, then separate the data frames for each zone.

The input data includes: 

* `CandLine.csv` which contains information on shared _candidate lines_ between zones
* `CandLineInt.csv` which contains information on internal _candidate lines_ within each zone
* `SharedEline.csv` which contains information on existing _shared lines_ between zones
* `Tran.csv` which contains information on existing transmission lines within each zone (i.e. as defined in the cited IEEE test systems)
* `Gen.csv` which contains information on the generators as defined in the cited IEEE test systems
* `Load.csv` which contains information on the loads as defined in the cited IEEE test systems

After loading the input data, construct several DataFrames for each zone, i.e. for each set of lines, generators, loads, etc.


In [57]:
# Read input files to set the problem

shared_cand = DataFrame(XLSX.readtable("../Input_Data/CandLine.csv", "Taul1", header=true)) #Dataframe of shared candidate lines
int_cand =  DataFrame(XLSX.readtable("../Input_Data/CandLineInt.csv", "Taul1", header=true)) #Dataframe of internal candidate lines
shared_ex =  DataFrame(XLSX.readtable("../Input_Data/SharedEline.csv", "Taul1", header=true)) #Dataframe of shared existing lines
int_ex = DataFrame(XLSX.readtable("../Input_Data/Tran.csv", "Taul1", header=true)) #Dataframe of internal existing lines
gen =  DataFrame(XLSX.readtable("../Input_Data/Gen.csv" , "Taul1", header=true)) #Dataframe of generators
load =  DataFrame(XLSX.readtable("../Input_Data/Load.csv", "Taul1", header=true)) #Dataframe of loads

Row,zoneNum,lNodeID,P_load1,P_load2,P_load3,P_load4
,Any,Any,Any,Any,Any,Any
1,1,2,-21.7,-10.85,-16.275,-5.425
2,1,3,-94.2,-47.1,-70.65,-23.55
3,1,4,-47.8,-23.9,-35.85,-11.95
4,1,5,-7.6,-3.8,-5.7,-1.9
5,1,6,-11.2,-5.6,-8.4,-2.8
6,1,9,-29.5,-14.75,-22.125,-7.375
7,1,10,-9,-4.5,-6.75,-2.25
8,1,11,-3.5,-1.75,-2.625,-0.875
9,1,12,-6.1,-3.05,-4.575,-1.525


In [58]:
# Functionally define dataframes for each element of each zone
# The left hand side is the singleton function that references the dataframe on the right hand side, e.g. l(1) is the load dataframe for zone 1

l(i) = load[load.zoneNum .== i, :] # load within zone i
g(i) = gen[gen.zoneNum .== i, :]   # generators within zone i
shared_c(i) = vcat(shared_cand[shared_cand.nodeZone1 .== i,:] , shared_cand[shared_cand.nodeZone2 .== i, :]) #shared candidate lines within zone i
int_c(i) = int_cand[int_cand.zoneNum .== i, :]   # number of internal candidate lines within zone i
shared_e(i) = vcat(shared_ex[shared_ex.nodeZone1 .== i,:] , shared_ex[shared_ex.nodeZone2 .== i, :]) #shared existing lines within zone i
int_e(i) =int_ex[int_ex.zoneNum .== i, :]       # internal existing lines within zone i
MC(i) = (g(i).C2 .* (g(i).PgMax .^ 2) .+ g(i).C1 .* g(i).PgMax .- g(i).C2 .*(g(i).PgMin .^ 2) .- g(i).C1 .* g(i).PgMin) ./ (g(i).PgMax .- g(i).PgMin) #Marginal cost of generators within zone i
bin_c(i) = (shared_cand.nodeZone1 .== i) + (shared_cand.nodeZone2 .== i) # A binary vector through which we can check if the shared candidate lines belong to zone i
bin_e(i) = (shared_ex.nodeZone1 .== i) + (shared_ex.nodeZone2 .== i) # A binary vector through which we can check if the shared existing lines belong to zone i

bin_e (generic function with 1 method)

## Build the Optimization Problem in JuMP

Initialize the JuMP model object by calling `Model(ENGING.Optimizer)` where `ENGINE` is your optimization engine of choice. 
Some requirements for the optimization engine include:
    1.
    2. 

After initializing the model, we will add the variables, constraints, and objective function. 
Decision variables for the model (and their variable type) include:
    * Generator dispatch (float)
    * Decision on candidate shared lines (binary)
    * Decision on candidate internal lines (binary)
    * Power flow (float) and phase angle (float) on candidate shared lines
    * Power flow (float) and phase angle (float) on candidate internal lines
    * Power flow (float) and phase angle (float) on existing shared lines
    * Power flow (float) and phase angle (float) on existing internal lines

The objective function is the `total_cost` to build new lines and dispatch generation. The cost is defined as:
\[EQUATION GOES HERE\]

The constraints include: 
\[EQUATIONS GO HERE\]
    

In [59]:
# initialize the JuMP model object and set the optimization engine
# solvers = [Gurobi.Optimizer, Clp.Optimzer, Cbc.Optimizer, HiGHS.Optimizer, Ipopt.Optimizer, CPLEX.Optimizer, Xpress.Optimizer]


Mod3 = Model(HiGHS.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS

In [7]:
node_number = 3;

In [60]:
# Add the decision variables to the model

# Decision variables for the power generation at each node
@variable(Mod3,0 <= gen_var_1[1:14]) # generation at node 1
@variable(Mod3,0 <= gen_var_2[1:30]) # generation at node 2
@variable(Mod3,0 <= gen_var_3[1:5]) # generation at node 3

# Decision variables for the candidate lines build/no build decision
@variable(Mod3,shared_line_decision_var[1:nrow(shared_cand)], Bin) #Decision variable for shared candidate lines
@variable(Mod3,int_line_decision_var_1[1:nrow(int_c(1))], Bin) #Decision variable for internal candidate lines of zone 1 (Binary)
@variable(Mod3,int_line_decision_var_2[1:nrow(int_c(2))], Bin) #Decision variable for internal candidate lines of zone 2 (Binary)
@variable(Mod3,int_line_decision_var_3[1:nrow(int_c(3))], Bin) #Decision variable for internal candidate lines of zone 3 (Binary)

# Decision variables for the power flow on the candidate lines
@variable(Mod3,shared_cand_flow[1:nrow(shared_cand)])  #Power flowing on shared candidate lines 

@variable(Mod3, int_cand_flow_1[1:nrow(int_c(1))]) #Power flowing on internal candidate lines of zone 1 
@variable(Mod3, int_cand_flow_2[1:nrow(int_c(2))]) #Power flowing on internal candidate lines of zone 2
@variable(Mod3, int_cand_flow_3[1:nrow(int_c(3))]) #Power flowing on internal candidate lines of zone 3

# Decision variables for the phase angle of the candidate lines
@variable(Mod3,0 <= shared_cand_angle[1:nrow(shared_cand),1:2]<= 2*pi) #Phase angle decision for shared candidate lines

@variable(Mod3,0 <= int_cand_angle_1[1:nrow(int_c(1)),1:2]<= 2*pi) #Phase angle decision for internal candidate lines of zone 1 
@variable(Mod3,0 <= int_cand_angle_2[1:nrow(int_c(2)),1:2]<= 2*pi) #Phase angle decision for internal candidate lines of zone 2
@variable(Mod3,0 <= int_cand_angle_3[1:nrow(int_c(3)),1:2]<= 2*pi) #Phase angle decision for internal candidate lines of zone 3 

# Decision variables for the power flow on the existing lines
@variable(Mod3, shared_ex_flow[1:nrow(shared_ex)])  #Power flowing on shared existing shared lines

@variable(Mod3, int_ex_flow_1[1:nrow(int_e(1))])  #Power flowing on internal existing lines of zone 1 
@variable(Mod3, int_ex_flow_2[1:nrow(int_e(2))])  #Power flowing on internal existing lines of zone 2
@variable(Mod3, int_ex_flow_3[1:nrow(int_e(3))])  #Power flowing on internal existing lines of zone 3

# Decision variables for the phase angle of the existing lines
@variable(Mod3,0 <= shared_ex_angle[1:nrow(shared_ex),1:2]<= 2*pi) #Phase angle for existing shared lines

@variable(Mod3,0 <= int_ex_angle_1[1:nrow(int_e(1)),1:2]<= 2*pi) #Phase angle decision for existing internal lines of zone 1
@variable(Mod3,0 <= int_ex_angle_2[1:nrow(int_e(2)),1:2]<= 2*pi) #Phase angle decision for existing internal lines of zone 2
@variable(Mod3,0 <= int_ex_angle_3[1:nrow(int_e(3)),1:2]<= 2*pi) #Phase angle decision for existing internal lines of zone 3


7×2 Matrix{VariableRef}:
 int_ex_angle_3[1,1]  int_ex_angle_3[1,2]
 int_ex_angle_3[2,1]  int_ex_angle_3[2,2]
 int_ex_angle_3[3,1]  int_ex_angle_3[3,2]
 int_ex_angle_3[4,1]  int_ex_angle_3[4,2]
 int_ex_angle_3[5,1]  int_ex_angle_3[5,2]
 int_ex_angle_3[6,1]  int_ex_angle_3[6,2]
 int_ex_angle_3[7,1]  int_ex_angle_3[7,2]

In [61]:
# Build the total_cost objective function
# todo fix the hard coded values for each zone and make it dynamic
# todo move this block to after the constraints

@expression(Mod3, total_cost , sum((gen_var_1[n] .* sum((g(1).gNodeID .== n) .* MC(1))) for n in 1:14)
    .+ sum((gen_var_2[n] .* sum((g(2).gNodeID .== n) .* MC(2))) for n in 1:30)
    .+ sum((gen_var_3[n] .* sum((g(3).gNodeID .== n) .* MC(3))) for n in 1:5)
    .+ sum(shared_line_decision_var[c] .* shared_cand.costPerCap[c] .* shared_cand.interestRate[c] 
            .*((1 + shared_cand.interestRate[c]) .^ shared_cand.lifeTime[c]) ./ (((1 + shared_cand.interestRate[c]) .^ shared_cand.lifeTime[c])-1) for c in 1:nrow(shared_cand))
    .+ sum(int_line_decision_var_1[c] .* int_c(1).costPerCap[c] .* int_c(1).interestRate[c] 
            .*((1 + int_c(1).interestRate[c]) .^ int_c(1).lifeTime[c]) ./ (((1 + int_c(1).interestRate[c]) .^ int_c(1).lifeTime[c])-1) for c in 1:nrow(int_c(1)))
    .+ sum(int_line_decision_var_2[c] .* int_c(2).costPerCap[c] .* int_c(2).interestRate[c] 
            .*((1 + int_c(2).interestRate[c]) .^ int_c(2).lifeTime[c]) ./ (((1 + int_c(2).interestRate[c]) .^ int_c(2).lifeTime[c])-1) for c in 1:nrow(int_c(2)))
    .+ sum(int_line_decision_var_3[c] .* int_c(3).costPerCap[c] .* int_c(3).interestRate[c] 
            .*((1 + int_c(3).interestRate[c]) .^ int_c(3).lifeTime[c]) ./ (((1 + int_c(3).interestRate[c]) .^ int_c(3).lifeTime[c])-1) for c in 1:nrow(int_c(3))))   

34.30293932 gen_var_1[1] + 55 gen_var_1[2] + 3.6 gen_var_2[1] + 3.15 gen_var_2[2] + 3.5 gen_var_2[13] + 3.667 gen_var_2[22] + 3.75 gen_var_2[23] + 4.375 gen_var_2[27] + 16.30293932 gen_var_3[1] + 37 gen_var_3[2] + 0.2410176361372405 shared_line_decision_var[1] + 0.19594024895900847 shared_line_decision_var[2] + 0.44297470440637654 shared_line_decision_var[3] + 0.6532451422553538 shared_line_decision_var[4] + 0.1533748593600621 shared_line_decision_var[5] + 0.34289543567826486 shared_line_decision_var[6] + 0.1533748593600621 int_line_decision_var_1[1] + 0.34289543567826486 int_line_decision_var_1[2] + 0.2519729832343877 int_line_decision_var_1[3] + 0.21553427385490934 int_line_decision_var_1[4] + 0.44297470440637654 int_line_decision_var_1[5] + 6.532451422553539 int_line_decision_var_1[6] + 1.5337485936006214 int_line_decision_var_1[7] + 3.673879667981409 int_line_decision_var_1[8] + 2.355399625886668 int_line_decision_var_1[9] + 2.400268049747854 int_line_decision_var_1[10] + 1.5337485

In [62]:
# Define constraints for Zone 1
# This includes nodal power balance constraints and upper and lower bound of connecting generators
# There are 14 buses in Zone 1

for n in 1:14
        # Power balance constraint for each node
    @constraint(Mod3, sum(g(1).gNodeID .== n) .* gen_var_1[n] .+ sum(l(1).P_load1 .* (l(1).lNodeID .== n)) .==
        sum((shared_cand.tNodeID1 .== n) .*bin_c(1) .* shared_cand_flow) .- sum((shared_cand.tNodeID2 .== n) .* bin_c(1) .* shared_cand_flow) .+
        sum((shared_ex.tNodeID1 .== n) .* bin_e(1) .* shared_ex_flow) .- sum((shared_ex.tNodeID2 .== n) .* bin_e(1) .* shared_ex_flow) .+
        sum((int_c(1).tNodeID1 .== n) .* int_cand_flow_1) .- sum((int_c(1).tNodeID2 .== n) .* int_cand_flow_1) .+
        sum((int_e(1).tNodeID1 .== n) .* int_ex_flow_1) .- sum((int_e(1).tNodeID2 .== n) .* int_ex_flow_1))
        #Lower limit for generation of each node
    @constraint(Mod3, sum(g(1).gNodeID .== n) .* gen_var_1[n] .<= sum((g(1).gNodeID .== n) .* g(1).PgMax))
        #Upper limit for generation of each node
    @constraint(Mod3, sum((g(1).gNodeID .== n) .* g(1).PgMin) .<= sum(g(1).gNodeID .== n) .* gen_var_1[n])
end

In [63]:
# Define constraints for Zone 2
# This includes nodal power balance constraints and upper and lower bound of connecting generators
# There are 30 buses in Zone 2

for n in 1:30
    # Power balance constraint for each node
    @constraint(Mod3, sum(g(2).gNodeID .== n) .* gen_var_2[n] .+ sum(l(2).P_load1 .* (l(2).lNodeID .== n)) .==
        sum((shared_cand.tNodeID1 .== n) .*bin_c(2) .* shared_cand_flow) .- sum((shared_cand.tNodeID2 .== n) .* bin_c(2) .* shared_cand_flow) .+
        sum((shared_ex.tNodeID1 .== n) .* bin_e(2) .* shared_ex_flow) .- sum((shared_ex.tNodeID2 .== n) .* bin_e(2) .* shared_ex_flow) .+
        sum((int_c(2).tNodeID1 .== n) .* int_cand_flow_2) .- sum((int_c(2).tNodeID2 .== n) .* int_cand_flow_2) .+
        sum((int_e(2).tNodeID1 .== n) .* int_ex_flow_2) .- sum((int_e(2).tNodeID2 .== n) .* int_ex_flow_2))
    
    # Lower limit for generation of each node
    @constraint(Mod3, sum(g(2).gNodeID .== n) .* gen_var_2[n] .<= sum((g(2).gNodeID .== n) .* g(2).PgMax))
    
    # Upper limit for generation of each node
    @constraint(Mod3, sum((g(2).gNodeID .== n) .* g(2).PgMin) .<= sum(g(2).gNodeID .== n) .* gen_var_2[n])
end

In [64]:
# Define constraints for  Zone 3
# This includes nodal power balance constraints and upper and lower bound of connecting generators
# There are 5 buses in Zone 3

for n in 1:5
    # Power balance constraint for each node
    @constraint(Mod3, sum(g(3).gNodeID .== n) .* gen_var_3[n] .+ sum(l(3).P_load1 .* (l(3).lNodeID .== n)) .==
        sum((shared_cand.tNodeID1 .== n) .*bin_c(3) .* shared_cand_flow) .- sum((shared_cand.tNodeID2 .== n) .* bin_c(3) .* shared_cand_flow) .+
        sum((shared_ex.tNodeID1 .== n) .* bin_e(3) .* shared_ex_flow) .- sum((shared_ex.tNodeID2 .== n) .* bin_e(3) .* shared_ex_flow) .+
        sum((int_c(3).tNodeID1 .== n) .* int_cand_flow_3) .- sum((int_c(3).tNodeID2 .== n) .* int_cand_flow_3) .+
        sum((int_e(3).tNodeID1 .== n) .* int_ex_flow_3) .- sum((int_e(3).tNodeID2 .== n) .* int_ex_flow_3))
    
    # Lower limit for generation of each node
    @constraint(Mod3, sum(g(3).gNodeID .== n) .* gen_var_3[n] .<= sum((g(3).gNodeID .== n) .* g(3).PgMax))
    
    # Upper limit for generation of each node
    @constraint(Mod3, sum((g(3).gNodeID .== n) .* g(3).PgMin) .<= sum(g(3).gNodeID .== n) .* gen_var_3[n])
end

In [65]:
# Deine constraints for the shared candidate lines
# The parameter big M is used to ensure equality constraints are satisfied when the candidate line is selected. 
# Choosing M is an art with some hueristic approaches. We want the smallest M that is still large enough to ensure the constraints are satisfied.
# For this problem, M should be related to the maximum power flow on the candidate line, assuming a 100 MW base power 

M = 500/100 
for c in 1:nrow(shared_cand)
    @constraint(Mod3,-M .* (1 .- shared_line_decision_var[c]) .<= shared_cand_flow[c] .- ((1 ./ shared_cand.reacT[c]) .* (shared_cand_angle[c,1] .- shared_cand_angle[c,2])))
        
    @constraint(Mod3, shared_cand_flow[c] .- ((1 ./ shared_cand.reacT[c]) .* (shared_cand_angle[c,1] .- shared_cand_angle[c,2])) .<= M .* (1 .- shared_line_decision_var[c]))
    
    # Upper bound of power flow flowing within candidate shared lines
    @constraint(Mod3, shared_cand_flow[c] .<= shared_line_decision_var[c] .*shared_cand.ptMax[c])
    
    # Lower bound of power flowing within the candidate shared lines
    @constraint(Mod3, -shared_cand.ptMax[c] .* shared_line_decision_var[c] .<= shared_cand_flow[c]) 
end

In [66]:
# Define constraints for existing shared lines
# These include power flow constraints and upper and lower bound of power flow on each line

for h in 1:nrow(shared_ex)
    @constraint(Mod3, shared_ex_flow[h] .== (1 ./ shared_ex.reacT[h]) .* (shared_ex_angle[h,1] .- shared_ex_angle[h,2]))
    @constraint(Mod3, shared_ex_flow[h] .<= shared_ex. ptMax[h])
    @constraint(Mod3, -shared_ex.ptMax[h] .<= shared_ex_flow[h])
end

In [67]:
# Define constraints for Zone 1 internal candidate lines
# These include power flow constraints and upper and lower bound of power flow on each line
# Note that we use the same parameter M as the shared candidate lines in this example. 
# In practice you may want to configure M differently for internal lines if they have different order of magnitude of power flow compared to shared lines

for c in 1:nrow(int_c(1))
    @constraint(Mod3,-M .* (1 .- int_line_decision_var_1[c]) .<= int_cand_flow_1[c] .- ((1 ./ int_c(1).reacT[c]) .* (int_cand_angle_1[c,1] .- int_cand_angle_1[c,2])))
    @constraint(Mod3, int_cand_flow_1[c] .- ((1 ./ int_c(1).reacT[c]) .* (int_cand_angle_1[c,1] .- int_cand_angle_1[c,2])) .<= M .* (1 .- int_line_decision_var_1[c]))
    @constraint(Mod3, int_cand_flow_1[c] .<= int_line_decision_var_1[c] .* int_c(1).ptMax[c])
    @constraint(Mod3, -int_c(1).ptMax[c] .*int_line_decision_var_1[c] .<= int_cand_flow_1[c])
end

In [68]:
# Define constraints for Zone 2 internal candidate lines
# These include power flow constraints and upper and lower bound of power flow on each line
# Note that we use the same parameter M as the shared candidate lines in this example. 
# In practice you may want to configure M differently for internal lines if they have different order of magnitude of power flow compared to shared lines

for c in 1:nrow(int_c(2))
    @constraint(Mod3,-M .* (1 .- int_line_decision_var_2[c]) .<= int_cand_flow_2[c] .- ((1 ./ int_c(2).reacT[c]) .* (int_cand_angle_2[c,1] .- int_cand_angle_2[c,2])))
    @constraint(Mod3, int_cand_flow_2[c] .- ((1 ./ int_c(2).reacT[c]) .* (int_cand_angle_2[c,1] .- int_cand_angle_2[c,2])) .<= M .* (1 .- int_line_decision_var_2[c]))
    @constraint(Mod3, int_cand_flow_2[c] .<= int_line_decision_var_2[c] .* int_c(2).ptMax[c])
    @constraint(Mod3, -int_c(2).ptMax[c] .*int_line_decision_var_2[c] .<= int_cand_flow_2[c])
end

In [69]:
# Define constraints for Zone 3 internal candidate lines
# These include power flow constraints and upper and lower bound of power flow on each line
# Note that we use the same parameter M as the shared candidate lines in this example. 
# In practice you may want to configure M differently for internal lines if they have different order of magnitude of power flow compared to shared lines

for c in 1:nrow(int_c(3))
    @constraint(Mod3,-M .* (1 .- int_line_decision_var_3[c]) .<= int_cand_flow_3[c] .- ((1 ./ int_c(3).reacT[c]) .* (int_cand_angle_3[c,1] .- int_cand_angle_3[c,2])))
    @constraint(Mod3, int_cand_flow_3[c] .- ((1 ./ int_c(3).reacT[c]) .* (int_cand_angle_3[c,1] .- int_cand_angle_3[c,2])) .<= M .* (1 .- int_line_decision_var_3[c]))
    @constraint(Mod3, int_cand_flow_3[c] .<= int_line_decision_var_3[c] .* int_c(3).ptMax[c])
    @constraint(Mod3, -int_c(3).ptMax[c] .*int_line_decision_var_3[c] .<= int_cand_flow_3[c])
end

In [70]:
# Define constraints for Zone 1 internal existing lines
# These include power flow constraints and upper and lower bound of power flow on each line

for h in 1:nrow(int_e(1))
    @constraint(Mod3, int_ex_flow_1[h] .== (1 ./ int_e(1).reacT[h]) .* (int_ex_angle_1[h,1] .- int_ex_angle_1[h,2]))
    @constraint(Mod3, int_ex_flow_1[h] .<= int_e(1).ptMax[h])
    @constraint(Mod3, -int_e(1).ptMax[h] .<= int_ex_flow_1[h])
end

In [71]:
# Define constraints for Zone 2 internal existing lines
# These include power flow constraints and upper and lower bound of power flow on each line

for h in 1:nrow(int_e(2))
    @constraint(Mod3, int_ex_flow_2[h] .== (1 ./ int_e(2).reacT[h]) .* (int_ex_angle_2[h,1] .- int_ex_angle_2[h,2]))
    @constraint(Mod3, int_ex_flow_2[h] .<= int_e(2).ptMax[h])
    @constraint(Mod3, -int_e(2).ptMax[h] .<= int_ex_flow_2[h])
    #@constraint(Mod3, (int_e(2).tNodeID1 .== 8) .* int_ex_angle_2[h,1] .== 0)
    #@constraint(Mod3, (int_e(2).tNodeID2 .== 8) .* int_ex_angle_2[h,2] .== 0)
end

In [92]:
# Define constraints for Zone 3 internal existing lines
# These include power flow constraints and upper and lower bound of power flow on each line

for h in 1:nrow(int_e(3))
    @constraint(Mod3, int_ex_flow_3[h] .== (1 ./ int_e(3).reacT[h]) .* (int_ex_angle_3[h,1] .- int_ex_angle_3[h,2]))
    @constraint(Mod3, int_ex_flow_3[h] .<= int_e(3).ptMax[h])
    @constraint(Mod3, -int_e(3).ptMax[h] .<= int_ex_flow_3[h])
end

In [109]:
# Collect the optimization problem and Solve

@objective(Mod3, Min, total_cost)
optimize!(Mod3)

Presolving model
267 rows, 372 cols, 911 nonzeros
186 rows, 186 cols, 593 nonzeros
168 rows, 169 cols, 549 nonzeros

Solving MIP model with:
   168 rows
   169 cols (35 binary, 0 integer, 0 implied int., 134 continuous)
   549 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0.2519729832    inf                  inf        0      0      0         0     0.0s
 R       0       0         0   0.00%   5309.792138     5969.659925       11.05%        0      0      0       138     0.0s
 C       0       0         0   0.00%   5318.196572     5821.938201        8.65%      248     26      0       213     0.0s
 L       0       0         0   0.00%   5318.435564     5318.596585        0.00%      330     39      0       245     0.2s

Solving report
  Status      

In [118]:
# Collect the results after the model has completed
# TODO: This should be a dataframe or dictionary


Centralized_with_angle_limit = Dict()   

if termination_status(Mod3) == MOI.OPTIMAL
    println("Optimal solution found.")
    Centralized_with_angle_limit["shared_line_decision"] = value.(shared_line_decision_var)
    Centralized_with_angle_limit["shared_cand_power"] = value.(shared_cand_flow)
    Centralized_with_angle_limit["shared_cand_phase_angle"] = value.(shared_cand_angle)
    Centralized_with_angle_limit["shared_ex_power"] = value.(shared_ex_flow)
    Centralized_with_angle_limit["shared_ex_phase_angle"] = value.(shared_ex_angle)
    Centralized_with_angle_limit["int_line_decision_1"] = value.(int_line_decision_var_1)
    Centralized_with_angle_limit["int_line_decision_2"] = value.(int_line_decision_var_2)
    Centralized_with_angle_limit["int_line_decision_3"] = value.(int_line_decision_var_3)
    Centralized_with_angle_limit["int_line_flow_1"] = value.(int_cand_flow_1)
    Centralized_with_angle_limit["int_line_flow_2"] = value.(int_cand_flow_2)
    Centralized_with_angle_limit["int_line_flow_3"] = value.(int_cand_flow_3)
    Centralized_with_angle_limit["generation_1"] = value.(gen_var_1)
    Centralized_with_angle_limit["generation_2"] = value.(gen_var_2)
    Centralized_with_angle_limit["generation_3"] = value.(gen_var_3)
    Centralized_with_angle_limit["obj_value"] = objective_value(Mod3)

    pprint(Centralized_with_angle_limit)
else
    println("Optimal solution not found.")
    value = JuMP.dual
end



Optimal solution found.
{
  "shared_cand_phase_angle" : Matrix{Float64}(),
  "shared_line_decision" : [1.0, 
                            1.0, 
                            0.0, 
                            1.0, 
                            1.0, 
                            1.0],
  "int_line_decision_1" : [0.0, 1.0, 
                           1.0, 0.0, 
                           0.0, 1.0, 
                           0.0, 0.0, 
                           0.0, 1.0],
  "int_line_flow_3" : [-1.7763568394002505e-14, 
                       -0.0],
  "shared_ex_phase_angle" : Matrix{Float64}(),
  "int_line_flow_2" : [-83.8876543014631, 
                       -0.0, -1.4210854715202004e-14, 
                       -0.0, -0.0, 
                       -31.62146606532253, 
                       -30.715412716952954, 
                       -0.0, -0.0, 
                       -32.67387055215592, 
                       -47.7305062045773, 
                       -0.0, -0.0, 
                       

     -0.0, 13.761720136141363],
  "shared_cand_power" : [89.30176838277129, 
                         -28.170665832046208, 
                         -0.0, 100.0, 
                         38.03380936549386, 
                         -35.612139172086565],
  "obj_value" : 5318.59658494212,
  "shared_ex_power" : [23.271056693257723, 
                       -0.0, 0.0, 
                       -0.0, 67.98177077193387, 
                       30.178811950047162, 
                       -20.530543391876098, 
                       6.806531259561455],
  "int_line_decision_2" : [1.0, 0.0, 
                           0.0, 0.0, 
                           0.0, 1.0, 
                           1.0, 0.0, 
                           0.0, 1.0, 
                           1.0, 0.0, 
                           0.0, 0.0, 
                           1.0, 0.0, 
                           0.0, 0.0],
  "int_line_decision_3" : [0.0, 0.0],
}